# Bedrock with LlamaIndex

## Setup

```python
%pip install llama-index-llms-bedrock
!pip install llama-index
```

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import os
import sys
import boto3
import botocore
from llama_index.llms.bedrock import Bedrock
import ipywidgets as widgets

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from labutils import bedrock, model_formatter, tools

profile_name = "genaidev"

In [3]:
_, bedrock_control_client = bedrock.get_bedrock_clients()

Create new client
  Using region: None
  Using profile: genaidev
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)
Create new client
  Using region: None
  Using profile: genaidev
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)


## Let's use 'complete' with one single prompt

In [4]:
user_input = """
            Write an email from Bob, Customer Service Manager, to the customer "John Doe" 
            who provided negative feedback on the service provided by our customer support 
            engineer"""

In [5]:
dropdown = tools.model_selection(bedrock_control_client)
display(dropdown)

Dropdown(description='Select LLM:', layout=Layout(width='300px'), options={'Claude': 'anthropic.claude-v2:1', …

In [10]:
# modelID = 'mistral.mistral-small-2402-v1:0'

In [14]:
response = Bedrock(
        model=dropdown.value,
        profile_name = profile_name).complete(user_input)

In [15]:
print(response)

Here is a sample email from Bob, the Customer Service Manager, to the customer "John Doe" who provided negative feedback on the service provided by the customer support engineer:

Subject: Addressing Your Feedback - Customer Service Improvement

Dear John Doe,

Thank you for taking the time to provide feedback on the service you received from our customer support team. I understand that your experience was not up to the high standards we strive for, and I appreciate you bringing this to our attention.

As the Customer Service Manager, I take all feedback seriously, as it helps us identify areas for improvement and ensure we are delivering the best possible experience for our customers. Please accept my sincere apologies for the inconvenience you faced.

I have reviewed the details of your interaction, and I would like to assure you that we are taking immediate steps to address the issues you encountered. Our customer support engineer has been counseled, and we will be providing additio

## Let's use 'chat' with one roles and system information

In [16]:
from llama_index.core.llms import ChatMessage
from llama_index.llms.bedrock import Bedrock

In [17]:
system = "You are a customer service manager."
user_input = """Create an apology email from the Service Manager Bob to John Doe in 
                response to the following feedback that was received from the customer:
                \n\nHello Bob,\nI am very disappointed with the recent experience I had 
                when I called your customer support.\nI was expecting an immediate call 
                back but it took three days for us to get a call back.\nThe first suggestion 
                to fix the problem was incorrect. Ultimately the problem was fixed after 
                three days.\nWe are very unhappy with the response provided and may consider 
                taking our business elsewhere."""

In [18]:
dropdown = tools.model_selection(bedrock_control_client)
display(dropdown)

Dropdown(description='Select LLM:', layout=Layout(width='300px'), options={'Claude': 'anthropic.claude-v2:1', …

In [19]:
messages = [
    ChatMessage(
        role="system", content = system
    ),
    ChatMessage(role="user", content = user_input),
]

response = Bedrock(
    model=dropdown.value, profile_name=profile_name).chat(messages)

In [20]:
print(response)

assistant: Here is a draft of the apology email from Service Manager Bob to John Doe:

Subject: Apology for Unsatisfactory Customer Service Experience

Dear John Doe,

I am writing to express my sincere apologies for the poor customer service experience you recently had when contacting our support team. I understand your frustration with the delayed response time, the incorrect initial suggestion, and the overall unsatisfactory resolution to your issue.

As the Service Manager, I take full responsibility for this unacceptable level of service. Your satisfaction is of the utmost importance to us, and we clearly fell short of meeting your expectations on this occasion. Please know that this is not the standard of service we strive to provide, and I am committed to addressing the issues that led to this situation.

I have thoroughly reviewed your case and spoken with the team to understand what went wrong. We have identified areas for improvement in our response protocols and staff traini

### Let's modify the instructions!

In [22]:
system = "You are a pirate with a colorful personality."

messages = [
    ChatMessage(
        role="system", content = system
    ),
    ChatMessage(role="user", content = user_input),
]

response = Bedrock(
    model=dropdown.value, profile_name=profile_name).chat(messages)

print(response)

assistant: Subject: Sincere Apologies for the Inconvenience

Dear John Doe,

I am writing to express my deepest apologies for the unsatisfactory experience you had with our customer support. As the Service Manager, I take full responsibility for the poor service you received, and I assure you that this is not the level of quality we strive for.

I understand your frustration with the delayed response and the initial incorrect suggestion to fix the problem. This is unacceptable, and I can assure you that we have taken immediate steps to address these issues.

Please accept my sincere apologies for the inconvenience and disruption this has caused you and your business. I understand the importance of timely and effective customer support, and I can assure you that we are committed to improving our processes to ensure that this does not happen again.

As a gesture of goodwill, I would like to offer you a 20% discount on your next service or repair. This is a small token of our appreciation

## Let's use streaming with 'stream_complete'

In [23]:
from llama_index.llms.bedrock import Bedrock

llm = Bedrock(model = dropdown.value, profile_name = profile_name)
response = llm.stream_complete(user_input)

for r in response:
    print(r.delta, end="")

Subject: Apology for Unsatisfactory Customer Service

Dear John Doe,

I am writing to express my sincere apologies for the unsatisfactory experience you had with our customer support. I understand your frustration with the delayed response and the initial incorrect suggestion to fix the problem.

As the Service Manager, I take full responsibility for this incident and the failure to meet your expectations. Your feedback is invaluable, and it will help us identify areas for improvement in our customer service processes.

Please accept my sincere apologies for the inconvenience and frustration you experienced. I assure you that we are committed to providing our customers with the highest level of service and support. We value your business and hope to have the opportunity to serve you better in the future.

If there is anything else I can do to address your concerns, please do not hesitate to let me know. I am available to discuss this matter further and work towards a resolution that me

## Let's use streaming with 'stream_chat'

In [24]:
from llama_index.llms.bedrock import Bedrock

system = "You are a customer service manager."
messages = [
    ChatMessage(
        role="system", content = system
    ),
    ChatMessage(role="user", content = user_input),
]

llm = Bedrock(model=dropdown.value, profile_name=profile_name)
resp = llm.stream_chat(messages)

for r in resp:
    print(r.delta, end="")

Here is a draft of the apology email from Service Manager Bob to John Doe:

Subject: Apology for Unsatisfactory Customer Service Experience

Dear John Doe,

I am writing to express my sincere apologies for the poor customer service experience you recently had when contacting our support team. I understand your frustration with the delayed response time, the incorrect initial suggestion, and the overall unsatisfactory resolution to your issue.

As the Service Manager, I take full responsibility for this unacceptable level of service. Your satisfaction is of the utmost importance to us, and we clearly fell short of meeting your expectations on this occasion. Please know that this is not the standard of service we strive to provide, and I am committed to addressing the issues that led to this situation.

I have thoroughly reviewed your case and spoken with the team to understand what went wrong. We have identified areas for improvement in our response protocols and staff training to ensur